In [72]:
import pandas as pd
df = pd.read_csv("../data/processed/data_model.csv")
df

,root_paper_id,ref_id,ref_title,published,ref_authors,in_text_citation_count,title_similarity,author_overlap,abstract_and_reference_title_similarity,standalone,first_quarter,second_quarter,third_quarter,fourth_quarter,reference_relevance
0,2,b0,The Market for 'Lemons': Quality Uncertainty a...,1970,['G Akerlof'],1,0.172840,0,0.252638,0,1,0,0,0,0
1,2,b2,Engineering Trust: Reciprocity in the Producti...,2013,"['G Bolton', 'B Greiner', 'A Ockenfels']",1,0.359281,0,0.248759,0,0,1,0,0,0
2,2,b3,Dispute Resolution or Escalation? The Strategi...,2017,"['G Bolton', 'B Greiner', 'A Ockenfels']",1,0.357143,0,0.292528,0,0,1,0,0,0
3,2,b4,Hiring Preferences in Online Labor Markets: Ev...,2017,"['J Chan', 'J Wang']",1,0.461538,0,0.248376,0,0,1,0,0,0
4,2,b5,The Sound of Silence in Online Feedback: Estim...,2008,"['C Dellarocas', 'C Wood']",3,0.302083,0,0.318214,0,1,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,46,b80,"E-commerce product recommendation agents: Use,...",2007. 2007,"['B Xiao', 'I Benbasat']",1,0.126984,0,0.159719,0,0,0,1,0,1
2237,46,b81,The new organizing logic of digital innovation...,2010,"['Y Yoo', 'O Henfridsson', 'K Lyytinen']",3,0.126761,0,0.338294,0,1,0,0,4,1
2238,46,b82,In the age of the smart machine: The future of...,1988,['S Zuboff'],3,0.058394,0,0.640541,0,1,1,1,0,0
2239,46,b83,Big other: Surveillance capitalism and the pro...,2015,['S Zuboff'],2,0.089552,0,0.408627,0,0,0,0,4,0


In [73]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['reference_relevance', 'published', 'ref_title','root_paper_id', 'ref_id', 'ref_authors'])
y = df['reference_relevance']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.25)

In [74]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

score = model.score(X_test, y_test)
report = classification_report(y_test, y_pred)
print(score,"\n", report)

0.7736185383244206 
               precision    recall  f1-score   support

           0       0.79      0.96      0.87       427
           1       0.59      0.16      0.26       134

    accuracy                           0.77       561
   macro avg       0.69      0.56      0.56       561
weighted avg       0.74      0.77      0.72       561



In [65]:
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': model.coef_[0]})
print(feature_importance.sort_values(by='importance', ascending=False))

                                    feature  importance
2                            author_overlap    0.388248
0                    in_text_citation_count    0.374936
1                          title_similarity    0.246388
3  abstract_and_reference_title_similarity     0.022086
7                             third_quarter    0.017734
4                                standalone   -0.088220
8                            fourth_quarter   -0.107704
5                             first_quarter   -0.112163
6                            second_quarter   -0.140031


In [66]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

In [67]:
svm_model = SVC(probability=True)  # Set probability=True to enable probability estimates for ROC AUC score calculation

svm_model.fit(X_train, y_train)

SVC(probability=True)

In [68]:
from sklearn.svm import SVC
import pandas as pd

# Initialize your SVM model
svm_model = SVC(kernel='linear')

# Train your SVM model
svm_model.fit(X_train, y_train)

# Predict using the SVM model
y_pred = svm_model.predict(X_test)
print("Accuracy:", svm_model.score(X_test, y_test))
print(classification_report(y_test, y_pred))

# Since your SVM model uses a linear kernel, extract feature weights
feature_importance_svm = pd.DataFrame({'feature': X.columns, 'importance': svm_model.coef_[0]})
print(feature_importance_svm.sort_values(by='importance', ascending=False))


Accuracy: 0.7771836007130125
              precision    recall  f1-score   support

           0       0.78      0.98      0.87       427
           1       0.66      0.14      0.23       134

    accuracy                           0.78       561
   macro avg       0.72      0.56      0.55       561
weighted avg       0.75      0.78      0.72       561

                                    feature  importance
2                            author_overlap    0.425916
5                             first_quarter    0.000172
6                            second_quarter    0.000103
8                            fourth_quarter    0.000101
7                             third_quarter    0.000071
4                                standalone    0.000044
3  abstract_and_reference_title_similarity     0.000014
1                          title_similarity   -0.000018
0                    in_text_citation_count   -0.000238


In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [70]:
# Initialize the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=0)

# Fit the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [71]:
# Predict the labels of the test set
y_pred = rf_model.predict(X_test)

# Print the accuracy
print("Accuracy:", rf_model.score(X_test, y_test))

# Print other metrics
print(classification_report(y_test, y_pred))
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': rf_model.feature_importances_})
print(feature_importance.sort_values(by='importance', ascending=False))

Accuracy: 0.732620320855615
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       427
           1       0.40      0.23      0.29       134

    accuracy                           0.73       561
   macro avg       0.59      0.56      0.56       561
weighted avg       0.69      0.73      0.71       561

                                    feature  importance
1                          title_similarity    0.377280
3  abstract_and_reference_title_similarity     0.358504
0                    in_text_citation_count    0.054868
5                             first_quarter    0.046554
2                            author_overlap    0.043118
8                            fourth_quarter    0.037579
6                            second_quarter    0.028623
4                                standalone    0.027636
7                             third_quarter    0.025837
